In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
from delta.tables import DeltaTable
from datetime import datetime
from shapely.geometry.base import BaseGeometry
from shapely import force_2d
import geopandas as gpd
from shapely.geometry import MultiPolygon

In [0]:
gcs_landing_zone = "/Volumes/land_topografisk-gdb_dev/external_dev/landing_zone/"
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev= "ai2025"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
log_table = "logs_processed_gdbs"
bronze_table= "utensnuplass_bronze"
layer = "ikke_snuplass"

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {bronze_table} (
   Shape_Length DOUBLE,
   Shape_Area DOUBLE,
   bbox STRING,
   id STRING,
   ingest_time TIMESTAMP,
   row_hash STRING
) USING DELTA
"""
spark.sql(q)

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {log_table} (
  gdb_name STRING,
  processed_time TIMESTAMP,
  num_inserted INT,
  num_updated INT,
  num_deleted INT
) USING DELTA
"""
spark.sql(q)

In [0]:
def log_processed_gdb(log_data: list):
    """
    Writes the processed gdb to the log table.
    """
    schema = StructType([
        StructField("gdb_name", StringType(), True),
        StructField("processed_time", TimestampType(), True),
        StructField("num_inserted", IntegerType(), True),
        StructField("num_updated", IntegerType(), True),
        StructField("num_deleted", IntegerType(), True)
        ])
    spark.createDataFrame(log_data, schema=schema).write.format("delta").mode("append").saveAsTable(log_table)

In [0]:
def check_for_new_gdbs() -> list:
    """
    Returns a lits of utensnuplass gdbs that have not been processed yet
    """
    all_gdbs =[
        f.path for f in dbutils.fs.ls(gcs_landing_zone)
        if f.path.endswith(".gdb/") 
    ]

    processed_gdbs_df=spark.read.table(log_table).select("gdb_name")
    processed_gdbs=[row["gdb_name"] for row in processed_gdbs_df.collect()]

    return [gdb for gdb in all_gdbs if gdb not in processed_gdbs]

In [0]:
def to_wkt_2d(geom):
    if geom and isinstance(geom, BaseGeometry):
        return force_2d(geom).wkt
    return None

In [0]:
def multipolygon_to_polygon(geom):
    if isinstance(geom, MultiPolygon):
        largest = None
        largest_area = -1
        for poly in geom.geoms:
            if poly.area > largest_area:
                largest = poly
                largest_area = poly.area
        return largest
    return geom

In [0]:
def write_to_sdf(gdb_path: str, gdb_name: str, layer: str) -> DataFrame:
    """
    Reads a .gdb layer and returns a minimal Spark DataFrame for bronze ingestion.
    """
    gdf = gpd.read_file(gdb_path, layer=layer)
    gdf = gdf.to_crs("EPSG:25833")
    gdf["geometry"] = gdf["geometry"].apply(multipolygon_to_polygon)
    gdf["wkt_geometry"] = gdf["geometry"].apply(to_wkt_2d)
    gdf = gdf.drop(columns=["geometry"])

    sdf = spark.createDataFrame(gdf).withColumnRenamed("wkt_geometry", "bbox")
    sdf = (
        sdf.withColumn("id", expr("uuid()"))
        .withColumn("source_file", lit(gdb_name))
        .withColumn("ingest_time", current_timestamp())
    )
    sdf = sdf.withColumn("row_hash", sha2(concat_ws("||", *sdf.columns), 256))

    return sdf

In [0]:
gdb_path = "/Volumes/land_topografisk-gdb_dev/external_dev/landing_zone/Snuplasser0108.gdb/"
gdb_name = "Snuplasser0108.gdb"
layer = "ikke_snuplass"

df = write_to_sdf(gdb_path, gdb_name, layer)

In [0]:
def write_delta_table(sdf: DataFrame):
    """
    Write delta table from spark dataframe.
    """
    if not spark.catalog.tableExists(bronze_table):
        sdf.write.format("delta").mode("overwrite").saveAsTable(bronze_table)
    else:
        delta_tbl = DeltaTable.forName(spark, bronze_table)
        delta_tbl.alias("target").merge(
                    source=sdf.alias("source"),
                    condition="target.row_hash = source.row_hash"
                ).whenMatchedUpdate(
                    condition="target.row_hash != source.row_hash",
                    set={col: f"source.{col}" for col in sdf.columns}
                ).whenNotMatchedInsert(
                    values={col: f"source.{col}" for col in sdf.columns}
                ).execute()

In [0]:
def write_to_delta_table(sdf: DataFrame, gdb_name: str):
    """
    Updates the delta table and logs the processed gdb.
    """
    table_exists = False
    if spark.catalog.tableExists(bronze_table):
        delta_tbl = DeltaTable.forName(spark, bronze_table)
        version_before = delta_tbl.history(1).select("version").collect()[0][0]
        table_exists = True

    write_delta_table(sdf)

    if table_exists:
        version_after = delta_tbl.history(1).select("version").collect()[0][0]
        if version_after > version_before:
            metrics = delta_tbl.history(1).select("operationMetrics").collect()[0][0]
            updated = int(metrics.get("numTargetRowsUpdated", 0))
            inserted = int(metrics.get("numTargetRowsInserted", 0))
            deleted = int(metrics.get("numTargetRowsDeleted", 0))
            print(f"Updated: {updated}, Inserted: {inserted}, Deleted: {deleted}")
        else:
            print("No new Delta version found after merge.")
    else:
        inserted, updated, deleted = sdf.count(), 0, 0
        print(f"Updated: {updated}, Inserted: {inserted}, Deleted: {deleted}")
    
    log_processed_gdb(log_data = [(gdb_name, datetime.now(), inserted, updated, deleted)])

In [0]:
def main():
    """
    Scans the landing zone for unprocessed .gdb files,
    extracts the specified layer, writes the data to the Delta bronze table,
    and logs the operation.
    """
    # Specify the layer name within the .gdb file
    layer = "ikke_snuplass"

    # Get list of unprocessed .gdb files
    gdbs = check_for_new_gdbs()


    # Loop over each .gdb file
    for gdb in gdbs:
        gdb_name = gdb.rstrip("/").split("/")[-1]        # extract file name
        gdb_path = gdb.removeprefix("dbfs:")             # fix file path
        print(f"\n🚧 Processing: {gdb_name}")

        try:
            # Convert the GDB layer to a Spark DataFrame
            sdf = write_to_sdf(gdb_path, gdb_name, layer)

            # Write data to Delta bronze table and log the operation
            write_to_delta_table(sdf, gdb_name)

            print(f"✅ Done: {gdb_name}")
        except Exception as e:
            print(f"❌ Error while processing {gdb_name}: {e}")
    df_bronze = spark.read.table(bronze_table)


In [0]:
main()